# Manipulacion de los datos

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('titanic-dataset.csv')


### Preprocesamiento y EDA

In [4]:
data.isna().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

In [34]:
df_clean = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1).copy()

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Cleaning NA Values
df_clean['FamilySize'] = df_clean['SibSp'] + df_clean['Parch'] + 1
df_clean['IsAlone'] = (df_clean['FamilySize'] == 1).astype(int)
df_clean.drop(columns=['SibSp', 'Parch'], inplace=True)


Survived        int64
Pclass          int64
Sex            object
Age           float64
Fare          float64
Embarked       object
FamilySize      int64
IsAlone         int64
dtype: object